In [41]:
import pandas as pd
import numpy as np
import pyomo.environ as pe
import random
from sklearn.preprocessing import MinMaxScaler

In [42]:
def bwm_opt_fn(n, best, worst, b_comparisons, w_comparisons):
    M = pe.ConcreteModel() # instancia modelo
    
    # Sets
    M.criteria_set = pe.Set(initialize = range(n))  #cria set de critérios no pyomo
    
    # Variables
    weights = M.weights = pe.Var(M.criteria_set, within = pe.NonNegativeReals)
    e = M.e = pe.Var(within = pe.NonNegativeReals)
    
    #Parameters
    M.best = pe.Param(initialize = best)
    M.worst = pe.Param(initialize = worst)
    M.b_comparisons = pe.Param(M.criteria_set, initialize = b_comparisons)
    M.w_comparisons = pe.Param(M.criteria_set, initialize = w_comparisons)
    
    #Objective
    M.obj = pe.Objective(rule = e, sense = pe.minimize)
    
    #Constraints
    M.Cons_1 = pe.ConstraintList()
    for j in M.criteria_set:
        M.Cons_1.add(weights[M.best] - M.b_comparisons[j] * weights[j] <= e)
        M.Cons_1.add(-weights[M.best] + M.b_comparisons[j] * weights[j] <= e)
    
    M.Cons_2 = pe.ConstraintList()
    for j in M.criteria_set:
        M.Cons_2.add(weights[j] - M.w_comparisons[j] * weights[M.worst] <= e)
        M.Cons_2.add(-weights[j] + M.w_comparisons[j] * weights[M.worst] <= e)
    
    M.Cons_3 = pe.Constraint(rule = lambda M: sum(M.weights[i] for i in M.criteria_set) == 1)  
    return M 


def Step (b_comparisons, w_comparisons, data):
    
    # inputs for the bwm function
    n_macro = 5
    best = 0
    worst = 1
    
    # run bwm to get the weights of the criteria (macro) 
    M = bwm_opt_fn(n_macro, best, worst, b_comparisons, w_comparisons)
    pe.SolverFactory('cplex').solve(M)
    
    # get the weights from the variables  
    var_list = [pe.value(v) for v in  M.component_data_objects([pe.Var])]
    weight_prof_macro = var_list[0]
    weights_cost_criteria = np.array(var_list[1:-1])
    
    # Now calculate get the final weight vector considering the 3 profitability criteria 
    weights_prof_micro = np.array([0.11111111, 0.22222222, 0.66666667])
    weights_prof = weights_prof_micro * weight_prof_macro
    
    # Concatenate the profitabilty weights with the weights that came from the other criteria
    weights = np.concatenate([weights_prof, weights_cost_criteria])

    # pandas DataFrame to numpy array
    X = np.array(data)
    # Apply MOORA normalization 
    X_norm = X/np.linalg.norm(X, axis=0)
    # Multiply by the weights
    X_wnorm = X_norm * weights

    # apply MOORA
    criteria_type = ["max", "max", "max", "min", "min", "min", "min"]
    m = X_wnorm.shape[0]
    n = X_wnorm.shape[1]

    values = np.zeros(m)
    for i in range (m):
        for j in range (n):
            if criteria_type[j] == "min":
                values[i] -= X_wnorm[i,j] 
            else:
                values[i] += X_wnorm[i,j]

    # Apply the final Normalization
    values = values.reshape(-1,1)
    final_result = MinMaxScaler().fit(values).transform(values)
    
    # Get a table of results
    table = pd.DataFrame(final_result, columns=["Value"], index=data.index)
    table["Position"] = table['Value'].rank(ascending=False)
    
    return table


In [28]:
data = pd.read_excel("data.xlsx", index_col=0)
sim_results = [] 

n_sim=1000
for sim in range (n_sim):
    b_comparisons = np.array([1, random.choice([9,8]), random.choice([3,4,5]), random.choice([4,5,6]), 1])
    w_comparisons = np.array([random.choice([9,8]),1,random.choice([5,6,7]),random.choice([4,5,6]),random.choice([9,8])])
    table = Step(b_comparisons, w_comparisons, data)   
    # Append to the simulation list of results
    sim_results.append(table)

In [35]:
values_sim = pd.concat([sim_results[s].Value for s in range(n_sim)], axis = 1)
positions_sim = pd.concat([sim_results[s].Position for s in range(n_sim)], axis = 1)

Post_analysis = pd.concat([values_sim.mean(axis = 1), 
                            values_sim.std(axis = 1),
                            positions_sim.mode(axis = 1), 
                            positions_sim.min(axis=1),
                            positions_sim.max(axis=1)], axis=1)

Post_analysis.columns = ["Mean Global Value", "Standard Deviation", "Position Mode", "Best Position", "Worst Position"]
#Post_analysis

In [43]:
# Original results
b_comparisons = np.array([1,9,4,5,1])
w_comparisons = np.array([9,1,6,5,9])
original_results = Step(b_comparisons, w_comparisons, data)
original_results

,Value,Position
Alternative,,
a1,0.000000,42.0
a2,0.617706,17.0
a3,0.303760,35.0
a4,0.278327,36.0
a5,0.506785,25.0
a6,0.192340,39.0
a7,0.832935,4.0
a8,0.781046,6.0
a9,0.227917,38.0


In [45]:
Result = pd.concat([table, Post_analysis], axis = 1)
Result

,Value,Position,Mean Global Value,Standard Deviation,Position Mode,Best Position,Worst Position
Alternative,,,,,,,
a1,0.000000,42.0,0.008302,1.632127e-02,42.0,40.0,42.0
a2,0.617706,17.0,0.634603,3.004946e-02,20.0,14.0,20.0
a3,0.303760,35.0,0.320859,2.286647e-02,35.0,34.0,36.0
a4,0.278327,36.0,0.294491,2.761400e-02,36.0,35.0,37.0
a5,0.506785,25.0,0.519490,2.248755e-02,25.0,24.0,28.0
a6,0.192340,39.0,0.192890,4.416977e-02,39.0,37.0,39.0
a7,0.832935,4.0,0.860194,4.453169e-02,4.0,3.0,5.0
a8,0.781046,6.0,0.804672,3.981397e-02,6.0,6.0,8.0
a9,0.227917,38.0,0.240943,2.864554e-02,38.0,38.0,39.0


In [44]:
# To get the weights in of the profitability criteria in the micro analysis

############################################################################
n = 3
best = 0
worst = 2
b_comparisons = np.array([1,3,6])
w_comparisons = np.array([6,2,1])

############################################################################
M = bwm_opt_fn(n, best, worst, b_comparisons, w_comparisons)
pe.SolverFactory('cplex').solve(M)
var_list = [pe.value(v) for v in  M.component_data_objects([pe.Var])]
weights_prof_micro = np.array(var_list[:-1]) 
weights_prof_micro = np.array([weights_prof_micro[2], weights_prof_micro[1], weights_prof_micro[0]])
weights_prof_micro

array([0.11111111, 0.22222222, 0.66666667])